<a href="https://colab.research.google.com/github/dldmstj0531/GEC/blob/main/notebooks/EDA/EDA_BEA19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BEA2019(W&I+LOCNESS)**

In [ ]:
!pip install -q koreanize-matplotlib

In [ ]:
!pip install python-Levenshtein seaborn nltk sacremoses pandas tqdm

In [ ]:
import warnings, os, json, ast, math, pathlib, shutil
import pandas as pd
import nltk
import Levenshtein  # 문자열 편집 거리 계산
from collections import Counter
import difflib  # Diff 분석
from tqdm.auto import tqdm  # 진행률 표시

# 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# pandas 출력 옵션 및 경고 설정
pd.set_option("display.max_columns", 120)
warnings.filterwarnings("ignore")

# tqdm의 pandas integration 활성화
tqdm.pandas()

# 마이너스 기호 깨짐 방지
sns.set(style="whitegrid")
plt.rcParams["axes.unicode_minus"] = False

# NLTK 토크나이저 다운로드 (최초 1회 필요)
nltk.download("punkt")
nltk.download("punkt_tab")

# 시각화 스타일 설정
sns.set(style="whitegrid")
plt.rcParams["axes.unicode_minus"] = False # 마이너스 기호 깨짐 방지

In [ ]:
# Colab 전용: tqdm, pandas, matplotlib 설치/업데이트
!pip -q install pandas tqdm matplotlib

# 구글 드라이브 마운트: EDA 결과물만 저장하고, 대형 TSV는 /content 에 임시 저장 권장
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

## 1. 데이터 준비

In [ ]:
%cd /content/drive/MyDrive/Projects/LikeLion/실전프로젝트02
%ls

In [ ]:
!tar -xvzf /content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness_v2.1.bea19.tar.gz
%ls

In [ ]:
%cd ./wi+locness/
%ls

In [ ]:
!head json/A.train.json

In [ ]:
!head m2/A.train.gold.bea19.m2

### JSON

In [ ]:
# JSON
file_paths = [
    "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/json/A.train.json",
    "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/json/B.train.json",
    "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/json/C.train.json",
]

all_data = []

for file_path in file_paths:
    if not os.path.exists(file_path):
        print(f"경고: 파일을 찾을 수 없습니다 - {file_path}")
        continue

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            for line_num, line in enumerate(f, 1):
                try:
                    data = json.loads(line)
                    all_data.append(data)
                except json.JSONDecodeError as e:
                    print(f"경고: JSON 파싱 오류 건너<0xEB><0>뜁니다 (파일: {file_path}, 라인: {line_num}): {e}")
                    continue
    except Exception as e:
        print(f"파일 읽기 중 오류 발생 {file_path}: {e}")
        continue


# all_data 확인
df_json = pd.DataFrame(all_data)
print(df_json.shape)
df_json.head()

In [ ]:
# JSON 저장 경로
save_path = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/bea19_json.csv"

# JSON csv 저장
df_json.to_csv(save_path, index=False)

# JSON 확인
if os.path.exists(save_path):
    print(f"파일 저장 확인: {save_path}")

### M2

In [ ]:
# M2
def parse_m2_file(path):
    rows = []
    sent_id = -1
    cur_sentence = None

    # m2는 문장 블록이 빈 줄로 구분됨
    def flush_sentence():
        # 필요시 문장 단위 후처리용 훅
        pass

    # 인코딩 문제 시 utf-8 → latin-1 폴백
    try_encodings = ["utf-8", "utf-8-sig", "latin-1"]
    for enc in try_encodings:
        try:
            with open(path, "r", encoding=enc) as f:
                lines = f.readlines()
            break
        except UnicodeDecodeError:
            continue
    else:
        raise UnicodeDecodeError("읽기 실패", b"", 0, 1, "모든 후보 인코딩 실패")

    for line in lines:
        line = line.rstrip("\n")
        if not line:
            flush_sentence()
            continue

        if line.startswith("S "):
            sent_id += 1
            cur_sentence = line[2:].strip()
        elif line.startswith("A "):
            if cur_sentence is None:
                # S 없이 A가 먼저 오는 비정상 라인 방어
                continue
            # "A 5 6|||R:OTHER|||- sized|||REQUIRED|||-NONE-|||0"
            try:
                # 앞의 위치정보 "A 5 6" 추출
                head, rest = line[2:].split("|||", 1)  # "5 6", "R:OTHER|||...|||0"
                head = head.strip()
                start_str, end_str = head.split()
                a_start, a_end = int(start_str), int(end_str)

                parts = rest.split("|||")
                # parts = [err_type, correction, status, comment, annotator, ...]
                # 일부 코퍼스는 길이가 5 미만/초과일 수 있어 안전 처리
                err_type   = parts[0].strip() if len(parts) > 0 else ""
                correction = parts[1].strip() if len(parts) > 1 else ""
                status     = parts[2].strip() if len(parts) > 2 else ""
                comment    = parts[3].strip() if len(parts) > 3 else ""
                annotator  = parts[4].strip() if len(parts) > 4 else ""

                rows.append({
                    "sent_id": sent_id,
                    "source": cur_sentence,     # 원문 문장
                    "a_start": a_start,         # 토큰 시작 인덱스 (포함)
                    "a_end": a_end,             # 토큰 끝 인덱스 (배타/포함은 코퍼스 정의에 따름; 보통 끝은 배타)
                    "err_type": err_type,       # 예: R:OTHER, M:PREP, U:PREP 등
                    "correction": correction,   # 수정안(빈 문자열일 수 있음)
                    "status": status,           # 예: REQUIRED / OPTIONAL 등
                    "comment": comment,         # 코멘트(없으면 -NONE-)
                    "annotator": annotator      # 주석자 ID (숫자 문자열)
                })
            except Exception as e:
                # 파싱 실패 라인은 그냥 건너뜀
                # print(f"[WARN] parse fail in {path}: {e}\n  line: {line}")
                continue
        else:
            # 다른 접두어는 무시(예: 'T ' 등이 있는 변형 코퍼스)
            continue

    return rows

In [ ]:
# 여러 m2 to DataFrame 만들기
file_paths = [
    "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/m2/A.train.gold.bea19.m2",
    "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/m2/B.train.gold.bea19.m2",
    "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/m2/C.train.gold.bea19.m2",
]

all_rows = []
for fp in file_paths:
    if not os.path.exists(fp):
        print(f"경고: 파일을 찾을 수 없습니다 - {fp}")
        continue
    rows = parse_m2_file(fp)
    # 원하면 파일 구분용 컬럼 추가
    for r in rows:
        r["m2_file"] = os.path.basename(fp)
    all_rows.extend(rows)

df_m2 = pd.DataFrame(all_rows, columns=[
    "m2_file", "sent_id", "source", "a_start", "a_end",
    "err_type", "correction", "status", "comment", "annotator"
])

print(df_m2.shape)
df_m2.head()

In [ ]:
# M2 저장 경로
save_path = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/bea19_m2.csv"

# M2 csv 저장
df_m2.to_csv(save_path, index=False)

# M2 확인
if os.path.exists(save_path):
    print(f"파일 저장 확인: {save_path}")

### src/tgt

In [ ]:
src_path = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/src_tgt/ABC.train.src"
tgt_path = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/wi+locness/src_tgt/ABC.train.tgt"


# 줄 단위로 읽기 (빈 줄도 포함해서)
with open(src_path, encoding="utf-8") as f:
    src_lines = [l.rstrip("\n") for l in f]

with open(tgt_path, encoding="utf-8") as f:
    tgt_lines = [l.rstrip("\n") for l in f]

print(f"src: {len(src_lines)} lines")
print(f"tgt: {len(tgt_lines)} lines")

# 2) 빈 줄 인덱스 찾기
src_empty_idx = [i for i, line in enumerate(src_lines) if line.strip() == ""]
tgt_empty_idx = [i for i, line in enumerate(tgt_lines) if line.strip() == ""]

print(f"src 빈 줄 개수: {len(src_empty_idx)}")
print(f"tgt 빈 줄 개수: {len(tgt_empty_idx)}")

# 3) tgt가 빈 줄인 곳의 src/tgt 내용 같이 보기
print("\n=== tgt가 빈 줄인 위치들 ===")
for i in tgt_empty_idx:
    src_val = src_lines[i]
    tgt_val = tgt_lines[i]
    print(f"[{i}]")
    print(f"  src: {repr(src_val)}")
    print(f"  tgt: {repr(tgt_val)}")

# # 빈 tgt를 src로 대체
# fixed_tgt_lines = [t if t.strip() != "" else s for s, t in zip(src_lines, tgt_lines)]

# # 검증
# empties_after = sum(1 for t in fixed_tgt_lines if t.strip() == "")
# print("empties_after:", empties_after)  # 0 이어야 정상
# assert len(src_lines) s== len(fixed_tgt_lines)

# # 빈 줄이 있는 위치 찾기
# src_empty_idx = [i for i, line in enumerate(src_lines) if line.strip() == ""]
# tgt_empty_idx = [i for i, line in enumerate(tgt_lines) if line.strip() == ""]

df = pd.DataFrame({"noise": src_lines, "clean": tgt_lines})
print(df.shape)
print(df.isnull().sum())
df.head()

In [ ]:
# SRC/TGT 저장 경로
save_path = "/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/bea19_train.csv"

# SRC/TGT csv 저장
df.to_csv(save_path, index=False)

# SRC/TGT 확인
if os.path.exists(save_path):
    print(f"파일 저장 확인: {save_path}")

## 2. EDA(json)

### 기본 통계

In [ ]:
df_json = pd.read_csv("/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/bea19_json.csv")
df_json.head()

In [ ]:
print(df_json.shape)
print(df_json["id"].value_counts().sum())

### `text` 단어 수

In [ ]:
df_json["text_word_len"] = df_json["text"].astype(str).str.split().str.len()
display(df_json.head())

# 문장 길이 시각화
plt.figure(figsize=(10, 5))
sns.histplot(df_json["text_word_len"], kde=True, bins=50)
plt.title("BEA-19 (JSON): 문장 길이(단어 수) 분포")
plt.xlabel("단어 수")
plt.ylabel("빈도수")
plt.xlim(0, 200)
plt.show()

# 평균 및 최대 문장 길이 출력
print(f"평균 문장 길이: {df_json['text_word_len'].mean():.2f}")
print(f"최대 문장 길이: {df_json['text_word_len'].max()}")
print(f"문장 길이 상위 95% 지점: {df_json['text_word_len'].quantile(0.95):.2f}")

### `edits` 개수

In [ ]:
print(df_json["text"].iloc[0])
df_json["edits"].iloc[0]

In [ ]:
# 각 문장당 edit(수정) 개수 계산
def count_edits_fixed(ed):
    # NaN/None
    if ed is None or (isinstance(ed, float) and math.isnan(ed)):
        return 0

    # 문자열(JSON 또는 파이썬 리터럴) -> 파싱
    if isinstance(ed, str):
        s = ed.strip()
        try:
            ed = json.loads(s)
        except Exception:
            try:
                ed = ast.literal_eval(s)
            except Exception:
                return 0

    # 이미 [[score, [[s,e,repl], ...]]]
    if (isinstance(ed, (list, tuple)) and ed
        and isinstance(ed[0], (list, tuple)) and len(ed[0]) > 1
        and isinstance(ed[0][1], (list, tuple))):
        inner = ed[0][1]
        return sum(1 for x in inner if isinstance(x, (list, tuple)) and len(x) == 3)

    # 이미 [[s,e,repl], ...] 형태
    if (isinstance(ed, (list, tuple)) and ed
        and all(isinstance(x, (list, tuple)) and len(x) == 3 for x in ed)):
        return len(ed)

    return 0

df_json["num_edits"] = df_json["edits"].apply(count_edits_fixed)
df_json.head()

In [ ]:
# 문장당 Edit 개수 분포 시각화
plt.figure(figsize=(10, 5))
edit_counts_vis = df_json["num_edits"].value_counts().sort_index()
edit_counts_vis = edit_counts_vis[edit_counts_vis.index <= 20]
sns.barplot(x=edit_counts_vis.index, y=edit_counts_vis.values)
plt.title("BEA-19 (JSON): 문장당 Edit 개수 분포 (상위 20개)")
plt.xlabel("문장당 Edit 개수")
plt.ylabel("빈도수")
plt.show()

# 평균 Edit 개수 및 Edit가 있는 문장 비율 출력
print(f"평균 Edit 개수: {df_json['num_edits'].mean():.2f}")
print(f"Edit가 하나 이상 있는 문장 비율: {(df_json['num_edits'] > 0).mean()*100:.2f}%")
print(f"Edit가 없는 문장 (No-Op) 비율: {(df_json['num_edits'] == 0).mean()*100:.2f}%")

[해석]

- 많은 오류를 포함
- 하나의 문장 내에서 많은 오류를 동시에 탐지하고 수정해야 함
    - `Seq2Seq(T5, BART)` 모델 적합성
- 수정없는 데이터 비율이 낮기 때문에 모델이 올바른 문장까지 불필요하게 수정하려는 경향이 보일 수도 있음.

## 2. EDA(SRC/TGT)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Projects/LikeLion/실전프로젝트02/bea19_train.csv")
df.head()

### 오류 존재 및 빈도 분석

In [ ]:
# 'noisy'와 'clean'이 다른 경우 True
df["is_corrected"] = (df["noise"] != df["clean"])

correction_ratio = df["is_corrected"].mean()

print(f"전체 샘플 중 수정된 샘플의 비율: {correction_ratio:.2%}")

### 오류 수정 규모 (편집 거리) 분석

In [ ]:
# ---------------------------------
# 2-1. 문자(Character) 레벨 편집 거리
# ---------------------------------
df["char_edit_distance"] = df.apply(
    lambda row: Levenshtein.distance(str(row["noise"]), str(row["clean"])),
    axis=1
)

print("\n[문자 레벨 편집 거리 통계]")
# 오류가 있는 샘플들만 통계 확인
print(df[df["is_corrected"]]["char_edit_distance"].describe())

In [ ]:
# ---------------------------------
# 2-2. 토큰(Word) 레벨 편집 거리
# ---------------------------------

def get_token_edit_distance(row):
    noise_tokens = nltk.word_tokenize(str(row["noise"]))
    clean_tokens = nltk.word_tokenize(str(row["clean"]))
    return nltk.edit_distance(noise_tokens, clean_tokens)

# tqdm을 사용하여 apply 진행 상황 확인
tqdm.pandas(desc="Token Edit Distance 계산 중")
df["token_edit_distance"] = df.progress_apply(get_token_edit_distance, axis=1)

print("\n[토큰 레벨 편집 거리 통계]")
# 오류가 있는 샘플들만 통계 확인
print(df[df["is_corrected"]]["token_edit_distance"].describe())

In [ ]:
# ---------------------------------
# 2-3. 시각화
# ---------------------------------
print("\n[편집 거리 분포 시각화 (수정된 샘플 대상)]")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 문자 편집 거리 (100 이상은 하나로 묶음)
sns.histplot(df[df["is_corrected"]]["char_edit_distance"].clip(upper=100), bins=50, ax=axes[0])
axes[0].set_title("Character Edit Distance Distribution (Corrected Samples)")
axes[0].set_xlabel("Char Edit Distance (Clipped at 100)")

# 토큰 편집 거리 (20 이상은 하나로 묶음)
sns.histplot(df[df["is_corrected"]]["token_edit_distance"].clip(upper=20), bins=20, ax=axes[1])
axes[1].set_title("Token Edit Distance Distribution (Corrected Samples)")
axes[1].set_xlabel("Token Edit Distance (Clipped at 20)")

plt.tight_layout()
plt.show()

- 히스토그램이 1~3과 같이 낮은 값에 몰려있다면, 대부분 간단한 수정(오타, 관사)
- 꼬리가 길거나 분포가 넓다면(e.g., 10 이상), 문장 구조를 바꾸는 복잡한 수정이 많다?

### 오류 유형 분석 (Token Diff)

In [ ]:
# Counter 객체 초기화
deleted_words = Counter()
added_words = Counter()
replaced_pairs = Counter()

# 수정된 샘플들만 순회
corrected_df = df[df["is_corrected"]]

for _, row in tqdm(corrected_df.iterrows(), total=corrected_df.shape[0], desc="Diff 패턴 분석 중"):
    noisy_tokens = nltk.word_tokenize(str(row["noise"]))
    clean_tokens = nltk.word_tokenize(str(row["clean"]))

    # difflib.SequenceMatcher를 사용하여 차이점 분석
    matcher = difflib.SequenceMatcher(None, noisy_tokens, clean_tokens)

    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "delete":
            # 삭제된 토큰들 추가
            deleted_words.update(noisy_tokens[i1:i2])

        elif tag == "insert":
            # 추가된 토큰들 추가
            added_words.update(clean_tokens[j1:j2])

        elif tag == "replace":
            # 대체된 토큰 쌍 추가
            # (noisy_tokens[i1:i2], clean_tokens[j1:j2] 형태)
            # 여기서는 간단히 1:1 매칭만 가정 (실제로는 N:M일 수 있음)
            noisy_segment = " ".join(noisy_tokens[i1:i2])
            clean_segment = " ".join(clean_tokens[j1:j2])
            replaced_pairs.update([(noisy_segment, clean_segment)])

print("\n[가장 많이 삭제된 단어 Top 20]")
print(deleted_words.most_common(20))

print("\n[가장 많이 추가된 단어 Top 20]")
print(added_words.most_common(20))

print("\n[가장 많이 대체된 (Noisy -> Clean) 쌍 Top 20]")
print(replaced_pairs.most_common(20))

### 문장 길이와 오류 상관관계

In [ ]:
# 'noisy' 문장의 토큰 길이 계산
df["noisy_token_count"] = df["noise"].apply(lambda x: len(nltk.word_tokenize(str(x))))

# 문장 길이와 토큰 편집 거리 간의 상관관계 계산
correlation = df["noisy_token_count"].corr(df["token_edit_distance"])

print(f"문장 길이(토큰 수)와 토큰 편집 거리 간의 상관계수: {correlation:.4f}")

if correlation > 0.3:
    print("- 문장이 길수록 수정량이 많아지는 경향이 있다.")
    print("- 모델이 긴 문맥(long-range dependency)을 잘 처리하는지 확인이 필요.")
elif correlation < 0.1:
    print("- 문장 길이와 오류 발생은 큰 직접적 관계가 없을 수 있다.")


# 시각화 (샘플 수가 많으므로 2D 히스토그램 또는 jointplot(sample)이 유용)
print("\n[문장 길이 vs 토큰 편집 거리 시각화 (샘플링)]")
# 데이터가 너무 많으면 오래 걸리므로 5000개 샘플링
sample_df = df.sample(n=min(5000, len(df)))

sns.jointplot(
    data=sample_df,
    x="noisy_token_count",
    y="token_edit_distance",
    kind="hist", # 'scatter', 'kde', 'hex' 등으로 변경 가능
    xlim=(0, 100), # 적절히 조절
    ylim=(0, 20)   # 적절히 조절
)
plt.suptitle("Sentence Length vs. Token Edit Distance (Sampled)", y=1.02)
plt.show()